# 1 Define Functions to Interact with the Twitter API

In [1]:
import requests 
import json 
import pandas as pd

# imports the twitter_secrets python file in which we store the twitter API keys
from twitter_secrets import twitter_secrets as ts

# puts the bearer token in the request header
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers
        
# sets the rules on which tweets to retrieve    
def set_rules(headers, delete, bearer_token, rules):
    payload = {"add": rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    
# retrieves the current set of rules from the API     
def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

# tells the API to delete our current rule configuration 
def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))

# starts the stream, iterates through the lines of the response and for each line calls the save_tweets function
def get_stream(headers, set, bearer_token, expansions, fields):
    data = []
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream" + expansions + fields, headers=headers, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i = 0
    for response_line in response.iter_lines():
        i += 1
        if i == max_results:
            break
        else:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True))
            try:
                save_tweets(json_response)
            except (json.JSONDecodeError, KeyError) as err:
                # In case the JSON fails to decode, we skip this tweet
                print(f"{i}/{max_results}: ERROR: encountered a problem with a line of data... \n")
                continue

# appends information from tweets to a dataframe                
def save_tweets(tweet):
    print(json.dumps(tweet, indent=4, sort_keys=True))
    data = tweet['data']
    public_metrics = data['public_metrics']
    tweet_list.append([data['id'], data['author_id'], data['created_at'], data['text'], public_metrics['like_count']])


{"data": [{"id": "1346440522056003585", "value": "dog has:images", "tag": "dog pictures"}], "meta": {"sent": "2021-05-02T14:36:02.989Z"}}
{"meta": {"sent": "2021-05-02T14:36:04.226Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "cat has:images -grumpy", "tag": "cat pictures", "id": "1388864873803702274"}, {"value": "dog has:images", "tag": "dog pictures", "id": "1388864873803702275"}], "meta": {"sent": "2021-05-02T14:36:05.398Z", "summary": {"created": 2, "not_created": 0, "valid": 2, "invalid": 0}}}
200
{
    "data": {
        "author_id": "1355182694997622785",
        "created_at": "2021-05-02T14:35:56.000Z",
        "id": "1388864840089968646",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 1
        },
        "text": "RT @annadegirona1: vols fer un regal original? roses fetes a m\u00e0 - en ganxet - artesania\n\nhttps://t.co/DSh5Ppu5Ho\n\n roses 5 euros - ciste

{
    "data": {
        "author_id": "138654861",
        "created_at": "2021-05-02T14:35:58.000Z",
        "id": "1388864849975910404",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 0
        },
        "text": "This. https://t.co/UtD4UoXWB3"
    },
    "matching_rules": [
        {
            "id": 1388864873803702274,
            "tag": "cat pictures"
        }
    ]
}
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1388862106196381698",
                "3_1388862111170826245"
            ]
        },
        "author_id": "1380153896123969543",
        "created_at": "2021-05-02T14:35:59.000Z",
        "id": "1388864854530969600",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 2
        },
        "text": "RT @confusedtotem: Blessing the tl with my

,tweetid,author_id,created_at,text,like_count
0,1388864840089968646,1355182694997622785,2021-05-02T14:35:56.000Z,RT @annadegirona1: vols fer un regal original?...,0
1,1388864842119847938,1359719162415771648,2021-05-02T14:35:56.000Z,CAT FIGHT🐈⚔️ありがとうございました～！\nTwitterのほんわか系とはギャップ...,0
2,1388864843684499458,1288738449189732352,2021-05-02T14:35:57.000Z,RT @ROLDAOrg: Another horrific case discovered...,0
3,1388864843663364097,38265398,2021-05-02T14:35:57.000Z,RT @mikegalsworthy: The dog’s expression says ...,0
4,1388864843701174275,1472033731,2021-05-02T14:35:57.000Z,C1 (F) - PSG - Olivier Echouafni (PSG) après l...,0
5,1388864844607090691,137965196,2021-05-02T14:35:57.000Z,RT @19951230vVvVvVv: 채널 십오야 인스타 / 인스타 스토리\n\nh...,0
6,1388864846947504131,1167047966730752002,2021-05-02T14:35:58.000Z,RT @HkanhpaSadan: Myitkyina Journal's publicat...,0
7,1388864846855172098,1260687295411138560,2021-05-02T14:35:58.000Z,RT @linandhers: #โทโกะ #甚五 യ c̶u̶r̶s̶e̶ cat us...,0
8,1388864847094317063,114043600,2021-05-02T14:35:58.000Z,RT @Lone: ANCIENT ASTRONAUTS\nw/ LONE\n\nBack ...,0
9,1388864848361136129,591623368,2021-05-02T14:35:58.000Z,Fora Colau! Cartell de la manifestació contra ...,0


# 2 Subscribe to the Tweet Streaming Service

In [3]:
# the max number of tweets that will be returned
max_results = 20

# You can adjust the rules if needed
search_rules = [
    {"value": "dog has:images", "tag": "dog pictures", "lang": "en"},
    {"value": "cat has:images -grumpy", "tag": "cat pictures", "lang": "en"},
]

# defines the fields which we want to retrieve
tweet_fields = "?tweet.fields=attachments,author_id,created_at,public_metrics"

# we only retrieve the tweet object, but if we wanted to retrieve other objects (e.g., media), we would add them to the expansions string
expansions = ""
tweet_list = []

bearer_token = ts.BEARER_TOKEN
headers = create_headers(bearer_token)
rules = get_rules(headers, bearer_token)
delete = delete_all_rules(headers, bearer_token, rules)
set = set_rules(headers, delete, bearer_token, search_rules)
get_stream(headers, set, bearer_token, expansions, tweet_fields)

df = pd.DataFrame (tweet_list, columns = ['tweetid', 'author_id' , 'created_at', 'text', 'like_count'])
df

{"data": [{"id": "1388864873803702274", "value": "cat has:images -grumpy", "tag": "cat pictures"}, {"id": "1388864873803702275", "value": "dog has:images", "tag": "dog pictures"}], "meta": {"sent": "2021-05-02T14:38:14.580Z"}}
{"meta": {"sent": "2021-05-02T14:38:15.878Z", "summary": {"deleted": 2, "not_deleted": 0}}}
{"data": [{"value": "cat has:images -grumpy", "tag": "cat pictures", "id": "1388865425992294400"}, {"value": "dog has:images", "tag": "dog pictures", "id": "1388865425992294401"}], "meta": {"sent": "2021-05-02T14:38:17.171Z", "summary": {"created": 2, "not_created": 0, "valid": 2, "invalid": 0}}}
200
{
    "data": {
        "author_id": "1228785355077214208",
        "created_at": "2021-05-02T14:38:07.000Z",
        "id": "1388865388981592066",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 25
        },
        "text": "RT @Mr_UsmanAnsari: Famous British Pop Singer cat steve

{
    "data": {
        "author_id": "1113974608733114368",
        "created_at": "2021-05-02T14:38:11.000Z",
        "id": "1388865406903930897",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 0
        },
        "text": "Me but with my dog https://t.co/uXiQXLHKwn"
    },
    "matching_rules": [
        {
            "id": 1388865425992294401,
            "tag": "dog pictures"
        }
    ]
}
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1388856353163079684"
            ]
        },
        "author_id": "1267749145957134337",
        "created_at": "2021-05-02T14:38:11.000Z",
        "id": "1388865408074059781",
        "public_metrics": {
            "like_count": 0,
            "quote_count": 0,
            "reply_count": 0,
            "retweet_count": 262
        },
        "text": "RT @somemememeya: cat\n#\u30a2\u30fc\u30af\u30ca\u30a4\u30

,tweetid,author_id,created_at,text,like_count
0,1388865388981592066,1228785355077214208,2021-05-02T14:38:07.000Z,RT @Mr_UsmanAnsari: Famous British Pop Singer ...,0
1,1388865389883449345,824700720876290052,2021-05-02T14:38:07.000Z,RT @Cat_Kapow: 🆘Today #MakeADifference &amp; r...,0
2,1388865390277652480,935446244989603840,2021-05-02T14:38:07.000Z,RT @somemememeya: cat\n#アークナイツ https://t.co/eQ...,0
3,1388865391758303234,4785486634,2021-05-02T14:38:07.000Z,RT @ConsellxRepComp: Actualment el Consell per...,0
4,1388865393675186178,1156256884644597761,2021-05-02T14:38:08.000Z,RT @k_katertot: lol this dog needs to *log off...,0
5,1388865394165882880,469406310,2021-05-02T14:38:08.000Z,RT @3CC0__: i-Dog by McDonald's (2007) https:/...,0
6,1388865400503300101,4761946880,2021-05-02T14:38:10.000Z,RT @minuet_nenosuke: もふり待ちの開脚\n#猫 #マンチカン #ミヌエッ...,0
7,1388865402709504000,890751484945547264,2021-05-02T14:38:10.000Z,RT @mimiatiny: ‘Black Cat Nero' Performance vi...,0
8,1388865404987129858,1160282851750948864,2021-05-02T14:38:11.000Z,RT @Seiurus: Please remember me this way: enjo...,0
9,1388865405603573761,1246940543415767041,2021-05-02T14:38:11.000Z,RT @19951230vVvVvVv: 채널 십오야 인스타 / 인스타 스토리\n\nh...,0
